In [1]:
import pandas as pd
from ase import neighborlist
from ase.neighborlist import natural_cutoffs
import graph_tool as gt
from graph_tool import topology
import numpy as np
import tqdm
from scipy.sparse.csgraph import connected_components
import dask.dataframe as ddf
from dask.distributed import Client, LocalCluster

In [5]:
def get_nuclearity(row):
    """Function to get the nuclearity for each element in a surface"""
    elements = row.bulk_elements
    slab_atoms = row.slab_surface_object.surface_atoms
    replicated_slab_atoms = slab_atoms.repeat((2,2,1))
    
    # Grab connectivity matricies
    overall_connectivity_matrix = get_connectivity_matrix(slab_atoms)
    overall_connectivity_matrix_rep = get_connectivity_matrix(replicated_slab_atoms)
    
    # Grab surface atom idxs
    surface_indices = [idx for idx, tag in enumerate(slab_atoms.get_tags()) if tag == 1]
    surface_indices_rep = [idx for idx, tag in enumerate(replicated_slab_atoms.get_tags()) if tag == 1]
    
    # Iterate over atoms and assess nuclearity
    output_dict = {}
    for element in elements:
        connected_surface_atoms = [atom.symbol == element and atom.index in surface_indices for atom in slab_atoms]
        connected_surface_atoms_rep = [atom.symbol == element and atom.index in surface_indices_rep for atom in replicated_slab_atoms]
        
        if sum(connected_surface_atoms) == 0:
            output_dict[element] = {"nuclearity": 0, "nuclearities": []}

        else:
            hist = get_nuclearity_neighbor_counts(connected_surface_atoms, overall_connectivity_matrix)
            hist_rep = get_nuclearity_neighbor_counts(connected_surface_atoms_rep, overall_connectivity_matrix_rep)
            output_dict[element] = evaluate_infiniteness(hist, hist_rep)
    return output_dict

def get_nuclearity_neighbor_counts(connected_surface_atoms, connectivity_matrix):
    connectivity_matrix = connectivity_matrix[connected_surface_atoms, :]
    connectivity_matrix = connectivity_matrix[:, connected_surface_atoms]
    graph = gt.Graph(directed=False)
    graph.add_vertex(n = connectivity_matrix.shape[0])
    graph.add_edge_list(np.transpose(connectivity_matrix.nonzero()))
    labels, hist = topology.label_components(graph, directed=False)
    return hist
            

def evaluate_infiniteness(hist, hist_rep):
    print(hist)
    if max(hist) == max(hist_rep):
        return {"nuclearity": max(hist), "nuclearities": hist}
    elif max(hist) == 0.5 * max(hist_rep):
        return {"nuclearity": 'semi-finite', "nuclearities": hist}
    elif max(hist) == 0.25 * max(hist_rep):
        return {"nuclearity": 'infinite', "nuclearities": hist}
    else:
        return {"nuclearity": 'somewhat-infinite', "nuclearities": hist}
    
def get_connectivity_matrix(slab_atoms):
    # For initial atoms
    surface_indices = [idx for idx, tag in enumerate(slab_atoms.get_tags()) if tag == 1]
    cutOff = natural_cutoffs(slab_atoms)
    neighborList = neighborlist.NeighborList(cutOff, self_interaction=False, bothways=True)
    neighborList.update(slab_atoms)
    overall_connectivity_matrix = neighborList.get_connectivity_matrix()
    return overall_connectivity_matrix


In [6]:
df = pd.read_pickle('../processing/outputs/post_processed_catlas_data_20220701_vals_2.pkl')    

In [6]:
cluster = LocalCluster(n_workers=32, threads_per_worker=1)
client = Client(cluster)

/opt/conda/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41837 instead
  warnings.warn(


In [7]:
df_dask = ddf.from_pandas(df, npartitions = 128)


In [10]:
df_dask_mapped = df_dask.apply(get_nuclearity, axis=1, meta=("nuclearities", object)).compute()
df["nuclearity"] = df_dask_mapped
df.to_pickle("all_info_w_nuclearity.pkl")

In [ ]:
df

In [7]:
for row_e in df.iterrows():
    row = row_e[1]
    blep = get_nuclearity(row)
    break

[1 1 1 1]
[8]
